In [ ]:
import os
import pandas as pd
#from nltk import word_tokenize, pos_tag, ne_chunk
#import re
from mordecai import Geoparser

# Download NLP data for country extraction // make this modular by setting nltk data path
nltk.download('treebank')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('punkt') # Download corpora for GPE extraction
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columns

Using TensorFlow backend.


In [ ]:
data = pd.read_csv('../data/clean/titanic_clean.csv')

In [ ]:
#data['home.dest'].str.split(' / ', expand=True)[0].value_counts()

In [ ]:
data.head()

In [ ]:
# Create dictionary of Country:[strings]
# If row.`home.dest` is in map assign Country value

abbrev_map = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    "Alberta": "AB",
    "British Columbia": "BC",
    "Manitoba": "MB",
    "New Brunswick": "NB",
    "Newfoundland": "NL",
    "Northwest Territories": "NT",
    "Nova Scotia": "NS",
    "Nunavut": "NU",
    "Ontario": "ON",
    "Prince Edward Island": "PE",
    "Quebec": "PQ",
    "Saskatchewan": "SK",
    "Yukon": "YT"}

# Invert mappings
abbrev_map = {v: k for k, v in abbrev_map.items()}

# Apply mappings to replace state abbreviations
data['home.dest'] = data['home.dest'].str.split().apply(lambda x: ' '.join([abbrev_map.get(word, word) for word in x]))

In [ ]:
data['home.dest']

In [ ]:
geo = Geoparser(country_threshold=0.9)
inferred = geo.geoparse("Birkdale, England / Cleveland, Ohio")

In [ ]:
inferred

In [ ]:
tokens = word_tokenize(sentence)

In [ ]:
tokens

In [ ]:
pos_list = pos_tag(tokens)

In [ ]:
entities = ne_chunk(pos_list)

In [ ]:
print(entities)

In [ ]:
home[1][0] in countries['United States']

In [ ]:
#data['home.dest'].str.contains('Co')

In [ ]:
for country in countries:
    if df.str.contains(country[city]):
        df['dest.country'] = countries_dict[country]
        
        
        #for country in countries.keys():
 #   for row in data.iterrows():
  #      for index, destination in home.iteritems():
   #         for idx, word in enumerate(destination):
    #            if word in countries[country]:
     #               #print(index)
      #              data[row, 'country'] = countries[country]